## Setup 

In [2]:
library(readr)
library(purrr)
library(dplyr)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Grab the data
I would normally grab this data straight from [the source](https://www.fisheries.noaa.gov/alaska/commercial-fishing/alaska-groundfish-bottom-trawl-survey-data#additional-resources), however errors in the multibyte string of the zip files make that difficult.  It is easier to grab the csv files from our [github repo](https://github.com/pinskylab/OceanAdapt/tree/master/data_raw).

*Special fix*  
There is a comment that contains a comma in the 2014-2018 file that causes the delimiters to read incorrectly.

In [4]:
temp <- read_lines("https://github.com/pinskylab/OceanAdapt/blob/master/data_raw/ai2014_2018.csv?raw=true")
# replace the string that causes the problem
temp_fixed <- stringr::str_replace_all(temp, "Stone et al., 2011", "Stone et al. 2011")
# read the result in as a csv
temp_csv <- read_csv(temp_fixed)

The following block takes a list of urls that represent csv files and uses the R package purrr to iterate through reading in each file.  Then it adds in the file we fixed above, removes wonky rows and whitespace, and puts all of the data into one table.  It is expected that there will be a warning message about 13 parsing failures, which is the original read in of the extra header row before we fix it with the filter.

In [7]:
files <- as.list(
  c("https://github.com/pinskylab/OceanAdapt/blob/master/data_raw/ai1983_2000.csv?raw=true",
    "https://github.com/pinskylab/OceanAdapt/blob/master/data_raw/ai2002_2012.csv?raw=true", 
    "https://github.com/pinskylab/OceanAdapt/blob/master/data_raw/ai2014_2018.csv?raw=true"))

# exclude the strata file and the raw 2014-2016 data file which has been fixed in temp_csv
files <- files[-c(grep("2014", files))]

ai_data <- files %>% 
  map_dfr(read_csv) %>%
  rbind(temp_csv) %>% 
  filter(LATITUDE != "LATITUDE", !is.na(LATITUDE)) %>% 
  mutate(COMMON = str_trim(COMMON), 
         SCIENTIFIC = str_trim(SCIENTIFIC))

Parsed with column specification:
cols(
  LATITUDE = col_double(),
  LONGITUDE = col_double(),
  STATION = col_character(),
  STRATUM = col_double(),
  YEAR = col_double(),
  DATETIME = col_character(),
  WTCPUE = col_double(),
  NUMCPUE = col_double(),
  COMMON = col_character(),
  SCIENTIFIC = col_character(),
  SID = col_double(),
  BOT_DEPTH = col_double(),
  BOT_TEMP = col_double(),
  SURF_TEMP = col_double(),
  VESSEL = col_double(),
  CRUISE = col_double(),
  HAUL = col_double()
)

Parsed with column specification:
cols(
  LATITUDE = col_double(),
  LONGITUDE = col_double(),
  STATION = col_character(),
  STRATUM = col_double(),
  YEAR = col_double(),
  DATETIME = col_character(),
  WTCPUE = col_double(),
  NUMCPUE = col_double(),
  COMMON = col_character(),
  SCIENTIFIC = col_character(),
  SID = col_double(),
  BOT_DEPTH = col_double(),
  BOT_TEMP = col_double(),
  SURF_TEMP = col_double(),
  VESSEL = col_double(),
  CRUISE = col_double(),
  HAUL = col_double()
)

Warning mess

In [9]:
head(ai_data)

LATITUDE,LONGITUDE,STATION,STRATUM,YEAR,DATETIME,WTCPUE,NUMCPUE,COMMON,SCIENTIFIC,SID,BOT_DEPTH,BOT_TEMP,SURF_TEMP,VESSEL,CRUISE,HAUL
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
53.18833,-169.6667,271-51,794,1983,08/23/1983 22:00,8.5509,5.6851,arrowtooth flounder,Atheresthes stomias,10110,401,3.5,6.4,554,198301,108
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,1.9251,0.3439,skate unid.,Rajidae,400,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,7.5880,-9999.0000,magistrate armhook squid,Berryteuthis magister,79210,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.0406,0.0688,flathead sole,Hippoglossoides elassodon,10130,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.1092,0.2064,rock sole unid.,Lepidopsetta sp.,10260,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.0063,0.0688,searcher,Bathymaster signatus,20720,219,3.8,6.3,554,198301,261


## Connect to the database and add the data

In [14]:
db <- src_sqlite("oceanadapt.db", create = FALSE)

In [17]:
copy_to(db, ai_data, "aleutian_islands_surveys", temporary = FALSE)


Warning message in .local(conn, statement, ...):
“partial argument match of 'param' to 'params'”


Check that it worked

In [18]:
test <- db %>% 
tbl("aleutian_islands_surveys") %>% 
collect()

head(test)

LATITUDE,LONGITUDE,STATION,STRATUM,YEAR,DATETIME,WTCPUE,NUMCPUE,COMMON,SCIENTIFIC,SID,BOT_DEPTH,BOT_TEMP,SURF_TEMP,VESSEL,CRUISE,HAUL
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
53.18833,-169.6667,271-51,794,1983,08/23/1983 22:00,8.5509,5.6851,arrowtooth flounder,Atheresthes stomias,10110,401,3.5,6.4,554,198301,108
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,1.9251,0.3439,skate unid.,Rajidae,400,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,7.5880,-9999.0000,magistrate armhook squid,Berryteuthis magister,79210,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.0406,0.0688,flathead sole,Hippoglossoides elassodon,10130,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.1092,0.2064,rock sole unid.,Lepidopsetta sp.,10260,219,3.8,6.3,554,198301,261
53.22000,170.7133,11-51,213,1983,10/31/1983 23:00,0.0063,0.0688,searcher,Bathymaster signatus,20720,219,3.8,6.3,554,198301,261


Add strata table

In [ ]:
ai_strata <- read_csv("", col_types = cols(NPFMCArea = col_character(),
      SubareaDescription = col_character(),
      StratumCode = col_integer(),
      DepthIntervalm = col_character(),
      Areakm2 = col_integer()
    ))  %>% 
      select(StratumCode, Areakm2) %>% 
  mutate(stratum = StratumCode)